# Answering Business Questions Using SQL

We'll be using the Chinook database, which is provided as a SQLite database file.

The Chinook record store has just signed a deal with a new record label. We've been tasked with selecting the first three albums from a list of four that will be added to the store. All four albums are by artists that don't have any tracks in the store right now. We have the artist names and the genre of music they produce:

| Artist Name | Genre |
| --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Bites |

Let's read in the file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

# Overview of the Data

In [2]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting Albums to Purchase

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [3]:
%%sql

WITH
    usa_tracks AS
    (
        SELECT il.*
        FROM invoice_line il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id
        WHERE c.country = "USA"
    )

SELECT 
    g.name genre,
    SUM(usat.quantity) num_tracks_sold,
    ROUND(CAST(SUM(usat.quantity) AS Float)/(
                                        SELECT COUNT(*)
                                        FROM usa_tracks    
                                    ),4) percentage_sold
FROM genre g
INNER JOIN track t ON t.genre_id = g.genre_id
INNER JOIN usa_tracks usat ON usat.track_id = t.track_id
GROUP BY 1
ORDER BY 2 DESC

Done.


genre,num_tracks_sold,percentage_sold
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres make up only 17% of total sales in the USA, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

# Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [4]:
%%sql

WITH 
    customer_total AS
    (
        SELECT c.*,
        SUM(i.total) total
        FROM invoice i
        INNER JOIN customer c ON c.customer_id = i.customer_id
        GROUP BY 1
    )

SELECT
    e.employee_id,
    e.first_name || " " || e.last_name employee_name,
    e.hire_date,
    printf("%.2f", ROUND(SUM(ct.total),2)) total_sales
FROM customer_total ct
INNER JOIN employee e ON e.employee_id = ct.support_rep_id
GROUP BY 1
    

Done.


employee_id,employee_name,hire_date,total_sales
3,Jane Peacock,2017-04-01 00:00:00,1731.51
4,Margaret Park,2017-05-03 00:00:00,1584.00
5,Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

# Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. In particular, we have been directed to calculate data for each country on the:
* total number of customers
* total value of sales
* average value of sales per custoemr
* average order value

Since there are a number of countries with only one customer, we will group these customers as "Other."

In [5]:
%%sql

WITH
    country_or_other AS
    (
        SELECT 
            CASE
            WHEN (SELECT COUNT(*)
                  FROM customer
                  WHERE country = c.country) = 1 THEN "Other"
            ELSE c.country
            END AS new_country,
            c.customer_id,
            i.total,
            i.invoice_id
        FROM customer c
        INNER JOIN invoice i ON i.customer_id = c.customer_id
    )

SELECT
    country,
    tot_cust,
    tot_sales,
    avg_sales_per_cust,
    avg_order_value
FROM (
        SELECT
            coo.new_country country,
            COUNT(DISTINCT(coo.customer_id)) tot_cust,
            ROUND(SUM(coo.total), 2) tot_sales,
            ROUND(SUM(coo.total)/COUNT(DISTINCT(coo.customer_id)), 2) avg_sales_per_cust,
            ROUND(SUM(coo.total)/COUNT(coo.invoice_id), 2) avg_order_value,
            CASE
                WHEN coo.new_country = "Other" THEN 1
                ELSE 0
            END sort
        FROM country_or_other coo
        GROUP BY coo.new_country
        ORDER BY sort, tot_sales DESC
    )


Done.


country,tot_cust,tot_sales,avg_sales_per_cust,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


The following three countries have the highest average order value:
* Czech Republic
* United Kingdom
* India

There may be opportunities in these countries for ad campaigns. However, it's worth keeping in mind that the amount of data from each of these countries is relatively low. Hence, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. 
A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

# Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

* purchase a whole album
* purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management is currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

Note that since we're concerned with maximizing revenue, we can ignore 
* albums consisting of only a few tracks
* the possibility that customers manually select every track from an album

In [8]:
%%sql

WITH invoice_first_track AS
(
    SELECT 
        il.invoice_id,
        MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
)


SELECT 
    album_purchase,
    COUNT(invoice_id) num_invoices,
    CAST(COUNT(invoice_id) AS Float)/(SELECT COUNT(*)
                                      FROM invoice) percent_invoices
FROM(
    SELECT 
        ifs.*,
        CASE
        WHEN (
                SELECT t.track_id FROM track t
                WHERE t.album_id = (
                                       SELECT t2.album_id FROM track t2
                                       WHERE t2.track_id = ifs.first_track_id
                                   )
                EXCEPT

                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ifs.invoice_id
             ) IS NULL

             AND

             (
                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ifs.invoice_id

                EXCEPT

                SELECT t.track_id FROM track t
                WHERE t.album_id = (
                                       SELECT t2.album_id FROM track t2
                                       WHERE t2.track_id = ifs.first_track_id
                                   )
             ) IS NULL

             THEN 1
        ELSE 0
        END AS album_purchase
    FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,num_invoices,percent_invoices
0,500,0.8143322475570033
1,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, it might not be beneficial purchasing only select tracks from albums from record companies, since there is potential to lose about one-fifth of revenue.